# WSG-84 GPS data to local ENU
- Calculates ENU coordinates for "Raw Data.cvs" file in same folder
- Raw data should come from PhyPhox app GPS-measurement
- csv-file should be formatted as tab as delimeter and decimal comma

In [ ]:
import numpy as np
import pandas as pd
import pymap3d as pm

df = pd.read_csv("Raw Data.csv",sep="\t",decimal=",")
df = df.drop(['Altitude (m)','Direction (°)','Satellites'],axis=1)

df["East (m)"] = np.nan
df["North (m)"] = np.nan
df["Up (m)"] = np.nan

ref = df.mean(axis=0)
lat_ref = ref.iloc[1]
lon_ref = ref.iloc[2]
h_ref = ref.iloc[3]

for i, row in df.iterrows():
    lat = row['Latitude (°)']
    lon = row['Longitude (°)']
    h = row['Altitude WGS84 (m)']
    e, n, u = pm.geodetic2enu(lat, lon, h, lat_ref, lon_ref, h_ref)
    df.loc[i, "East (m)"] = e
    df.loc[i, "North (m)"] = n
    df.loc[i, "Up (m)"] = u

df.to_csv('GPSdata.csv', index=False)

In [ ]:
from ipyleaflet import Map, AntPath

center = (lat_ref, lon_ref)
m = Map(center=center, zoom=15)

path = []
for lat,lon in zip(df['Latitude (°)'],df['Longitude (°)']):
    path.append([lat,lon])

ant_path = AntPath(
    locations=path,
    dash_array=[1, 10],
    delay=2000,
    color='#7590ba',
    pulse_color='#3f6fba'
)

m.add_layer(ant_path)

display(m)